In [28]:
import numpy as np
import warnings
import sys
import traceback
from joblib import load
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
import logging
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suppress warnings
warnings.filterwarnings('ignore')

In [29]:
# Scikit-learn Classifiers
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    BaggingClassifier, 
    IsolationForest
)
from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    RidgeClassifierCV,
    SGDClassifier,
    Perceptron,
    PassiveAggressiveClassifier,
    LogisticRegressionCV,
    BayesianRidge,
    PassiveAggressiveRegressor,
    HuberRegressor,
    RANSACRegressor,
    TheilSenRegressor
)
from sklearn.svm import SVC, LinearSVC, NuSVC, OneClassSVM
from sklearn.neighbors import (
    KNeighborsClassifier,
    RadiusNeighborsClassifier,
    NearestCentroid,
    LocalOutlierFactor
)
from sklearn.naive_bayes import (
    GaussianNB,
    MultinomialNB,
    ComplementNB,
    BernoulliNB,
    CategoricalNB
)
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis
)
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier

In [30]:
LIBRARY_STATUS = {
    'xgboost': False,
    'lightgbm': False,
    'catboost': False,
    'keras': False,
    'ngboost': False,
    'river': False,
    'sktime': False,
    'torch': False,
    'tensorflow': False,
    'thundersvm': False,
    'cuml': False,
    'fastai': False,
    'autogluon': False,
    'hyperopt': False
}

# Standard boosting libraries
try:
    from xgboost import XGBClassifier
    LIBRARY_STATUS['xgboost'] = True
except ImportError:
    logger.warning("XGBoost not available")

try:
    from lightgbm import LGBMClassifier
    LIBRARY_STATUS['lightgbm'] = True
except ImportError:
    logger.warning("LightGBM not available")

try:
    from catboost import CatBoostClassifier
    LIBRARY_STATUS['catboost'] = True
except ImportError:
    logger.warning("CatBoost not available")

# Deep Learning frameworks
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, GRU
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
    LIBRARY_STATUS['keras'] = True
    LIBRARY_STATUS['tensorflow'] = True
except ImportError:
    logger.warning("TensorFlow/Keras not available")

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader
    LIBRARY_STATUS['torch'] = True
except ImportError:
    logger.warning("PyTorch not available")

try:
    from fastai.tabular.all import *
    LIBRARY_STATUS['fastai'] = True
except ImportError:
    logger.warning("FastAI not available")

# Advanced boosting and online learning
try:
    from ngboost import NGBClassifier
    LIBRARY_STATUS['ngboost'] = True
except ImportError:
    logger.warning("NGBoost not available")

try:
    import river
    from river import linear_model, tree, ensemble
    from river.linear_model import LogisticRegression, PassiveAggressiveClassifier
    from river.naive_bayes import NaiveBayes
    LIBRARY_STATUS['river'] = True
except ImportError:
    logger.warning("River not available")

# Time series specific
try:
    from sktime.classification.sklearn import SklearnClassifier
    from sktime.classification.interval_based import TimeSeriesForestClassifier
    LIBRARY_STATUS['sktime'] = True
except ImportError:
    logger.warning("Sktime not available")

# AutoML and optimization
try:
    from autogluon.tabular import TabularPredictor
    LIBRARY_STATUS['autogluon'] = True
except ImportError:
    logger.warning("AutoGluon not available")

try:
    from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
    LIBRARY_STATUS['hyperopt'] = True
except ImportError:
    logger.warning("Hyperopt not available")

In [31]:
import sys

print(f"Python Version: {sys.version}")
print(f"Pandas Version: {pd.__version__}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

Python Version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
Pandas Version: 2.2.3
TensorFlow Version: 2.18.0
Keras Version: 3.7.0


In [32]:
def create_keras_model(input_dim):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_dim, kernel_initializer='he_normal'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [33]:
class PyTorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, epochs=100, lr=0.001):
        self.input_dim = input_dim
        self.epochs = epochs
        self.lr = lr
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _create_model(self):
        return nn.Sequential(
            nn.Linear(self.input_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        ).to(self.device)

    def fit(self, X, y):
        X = torch.FloatTensor(X).to(self.device)
        y = torch.FloatTensor(y).unsqueeze(1).to(self.device)

        self.model = self._create_model()
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        for _ in range(self.epochs):
            self.model.train()
            optimizer.zero_grad()
            outputs = self.model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

        return self

    def predict(self, X):
        X = torch.FloatTensor(X).to(self.device)
        with torch.no_grad():
            self.model.eval()
            predictions = self.model(X)
        return (predictions.cpu().numpy() > 0.5).astype(int).ravel()

In [34]:
# try:
#     data = np.load('../Data/#1/processed_data.npz')
#     x_tr_resample = data['x_tr_resample']
#     y_tr_resample = data['y_tr_resample']
#     X_test = data['X_test']
#     y_test = data['y_test']
#     X_train = data['X_train']

#     norm = load('../Data/#1/power_transformer.joblib')
#     norm_train_feature = norm.fit_transform(X_train)
#     norm_test_feature = norm.transform(X_test)
# except Exception as e:
#     logger.error(f"Error loading data: {e}")
#     sys.exit(1)

x_tr_resample = pd.read_csv('../Data/sequential/n=7/X_train_smote.csv')
y_tr_resample = pd.read_csv('../Data/sequential/n=7/y_train_smote.csv')
X_test = pd.read_csv('../Data/sequential/n=7/X_test.csv')
y_test = pd.read_csv('../Data/sequential/n=7/y_test.csv')

In [37]:
classifiers = {
    # Scikit-learn Base Classifiers
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Extra Tree': ExtraTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Hist Gradient Boosting': HistGradientBoostingClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(),
    'Logistic Regression CV': LogisticRegressionCV(random_state=42),
    'Ridge Classifier': RidgeClassifier(random_state=42),
    'Ridge Classifier CV': RidgeClassifierCV(),
    'SGD Classifier': SGDClassifier(random_state=42),
    'Bayesian Ridge': BayesianRidge(),
    'Perceptron': Perceptron(random_state=42),
    'Passive Aggressive': PassiveAggressiveClassifier(random_state=42),
    'SVM (RBF Kernel)': SVC(random_state=42),
    'SVM (Linear Kernel)': SVC(kernel='linear', random_state=42),
    'SVM (Polynomial Kernel)': SVC(kernel='poly', random_state=42),
    'Linear SVM': LinearSVC(random_state=42),
    'Nu SVM': NuSVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Radius Nearest Neighbors': RadiusNeighborsClassifier(radius=1.0, outlier_label='most_frequent'),
    'Nearest Centroid': NearestCentroid(),
    'One-Class SVM': OneClassSVM(),
    'Calibrated Classifier': CalibratedClassifierCV(cv=5),
    'Bagging Classifier': BaggingClassifier(random_state=42),
    'Isolation Forest': IsolationForest(random_state=42),
    'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, novelty=True),

    # Naive Bayes Classifiers
    'Gaussian Naive Bayes': GaussianNB(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Complement Naive Bayes': ComplementNB(),
    'Bernoulli Naive Bayes': BernoulliNB(),
    'Categorical Naive Bayes': CategoricalNB(),

    # Advanced Classifiers
    'Multi-Layer Perceptron': MLPClassifier(random_state=42, max_iter=1000),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
    'Label Spreading': LabelSpreading(),
    'Label Propagation': LabelPropagation(),
    'Gaussian Process': GaussianProcessClassifier(random_state=42),

    'Dummy Classifier': DummyClassifier(strategy='most_frequent'),
    'Gaussian Process Regressor': GaussianProcessRegressor(),
    'Passive Aggressive Regressor': PassiveAggressiveRegressor(),
    'Huber Regressor': HuberRegressor(),
    'RANSAC Regressor': RANSACRegressor(),
    'Theil Sen Regressor': TheilSenRegressor(),
}

In [38]:
if LIBRARY_STATUS['xgboost']:
    classifiers.update({
        'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        'XGBoost DART': XGBClassifier(random_state=42, booster='dart', use_label_encoder=False),
    })

if LIBRARY_STATUS['lightgbm']:
    classifiers.update({
        'LightGBM': LGBMClassifier(random_state=42),
        'LightGBM DART': LGBMClassifier(boosting_type='dart', random_state=42),
    })

if LIBRARY_STATUS['catboost']:
    classifiers.update({
        'CatBoost': CatBoostClassifier(random_state=42, verbose=0),
        'CatBoost L2': CatBoostClassifier(random_state=42, loss_function='MultiClass', verbose=0),
    })

if LIBRARY_STATUS['keras']:
    classifiers.update({
        'Keras Neural Network': KerasClassifier(
            build_fn=lambda: create_keras_model(x_tr_resample.shape[1]),
            epochs=100,
            batch_size=32,
            verbose=0
        )
    })

if LIBRARY_STATUS['ngboost']:
    classifiers.update({
        'NGBoost Classifier': NGBClassifier(random_state=42)
    })

if LIBRARY_STATUS['torch']:
    classifiers.update({
        'PyTorch Neural Network': PyTorchClassifier(input_dim=x_tr_resample.shape[1])
    })

if LIBRARY_STATUS.get('river', False): 
    classifiers.update({
        'Passive Aggressive River': PassiveAggressiveClassifier(),
        'River Logistic Regression': LogisticRegression(),
        'River Naive Bayes': NaiveBayes(),
    })

# if LIBRARY_STATUS['sktime']:
#     classifiers.update({
#         'Time Series Forest': TimeSeriesForestClassifier(random_state=42),
#         'Sktime Sklearn Wrapper': SklearnClassifier(estimator=RandomForestClassifier(random_state=42)),
#     })

In [39]:
def evaluate_model(name, model, x_tr_resample, y_tr_resample, X_test, y_test):
    try:
        # Special handling for OneClassSVM and unsupervised models
        if name == 'One-Class SVM':
            model.fit(x_tr_resample)
            preds = model.predict(X_test)
            preds = [1 if p == 1 else 0 for p in preds]
            accuracy = accuracy_score(y_test, preds)
        else:
            # Model Training and Prediction
            model.fit(x_tr_resample, y_tr_resample)
            preds = model.predict(X_test)
            accuracy = accuracy_score(y_test, preds)
        
        # Print the model name and accuracy result
        print(f"{name} - Accuracy: {accuracy * 100:.2f}%")
        
        return accuracy
    except Exception as e:
        # Print model name and error message if exception occurs
        print(f"{name} - Error: {str(e)}")
        return 0


In [40]:
def cross_validate_model(name, model, x_tr_resample, y_tr_resample):
    try:
        # Cross-validation using StratifiedKFold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, x_tr_resample, y_tr_resample, cv=skf, scoring='accuracy')
        
        return cv_scores.mean(), cv_scores.std()
    except Exception as cv_error:
        logger.warning(f"Cross-validation failed for {name}: {cv_error}")
        return 0, 0


In [41]:
results = {}
for name, classifier in classifiers.items():
    results[name] = evaluate_model(name, classifier, x_tr_resample, y_tr_resample, X_test, y_test)

Decision Tree - Accuracy: 98.22%
Extra Tree - Accuracy: 98.43%
Random Forest - Accuracy: 99.14%
Gradient Boosting - Accuracy: 98.83%
Hist Gradient Boosting - Accuracy: 98.88%
Extra Trees - Accuracy: 99.14%
AdaBoost - Accuracy: 98.48%
Logistic Regression - Error: 'LogisticRegression' object has no attribute 'fit'
Logistic Regression CV - Accuracy: 97.11%
Ridge Classifier - Accuracy: 96.60%
Ridge Classifier CV - Accuracy: 96.60%
SGD Classifier - Accuracy: 97.61%
Bayesian Ridge - Error: Classification metrics can't handle a mix of binary and continuous targets
Perceptron - Accuracy: 95.53%
Passive Aggressive - Accuracy: 88.12%
SVM (RBF Kernel) - Accuracy: 98.53%
SVM (Linear Kernel) - Accuracy: 96.65%
SVM (Polynomial Kernel) - Accuracy: 95.28%
Linear SVM - Accuracy: 97.00%
Nu SVM - Accuracy: 94.82%
K-Nearest Neighbors - Accuracy: 97.92%
Radius Nearest Neighbors - Accuracy: 97.61%
Nearest Centroid - Accuracy: 73.74%
One-Class SVM - Accuracy: 45.81%
Calibrated Classifier - Accuracy: 96.95%
B

In [42]:
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

print("\n--- Ranking Model Berdasarkan Akurasi Uji ---")
for rank, (name, accuracy) in enumerate(sorted_results, 1):
    if accuracy > 0:
        print(f"{rank}. {name}: {accuracy * 100:.2f}%")


--- Ranking Model Berdasarkan Akurasi Uji ---
1. CatBoost: 99.29%
2. CatBoost L2: 99.19%
3. Random Forest: 99.14%
4. Extra Trees: 99.14%
5. Bagging Classifier: 98.98%
6. LightGBM: 98.93%
7. Hist Gradient Boosting: 98.88%
8. XGBoost: 98.88%
9. XGBoost DART: 98.88%
10. LightGBM DART: 98.88%
11. Gradient Boosting: 98.83%
12. Gaussian Process: 98.68%
13. Multi-Layer Perceptron: 98.63%
14. SVM (RBF Kernel): 98.53%
15. AdaBoost: 98.48%
16. Extra Tree: 98.43%
17. Decision Tree: 98.22%
18. Label Spreading: 98.22%
19. Label Propagation: 98.22%
20. K-Nearest Neighbors: 97.92%
21. Quadratic Discriminant Analysis: 97.77%
22. SGD Classifier: 97.61%
23. Radius Nearest Neighbors: 97.61%
24. Logistic Regression CV: 97.11%
25. Linear SVM: 97.00%
26. Calibrated Classifier: 96.95%
27. SVM (Linear Kernel): 96.65%
28. Ridge Classifier: 96.60%
29. Ridge Classifier CV: 96.60%
30. Linear Discriminant Analysis: 96.60%
31. Gaussian Naive Bayes: 95.89%
32. Perceptron: 95.53%
33. SVM (Polynomial Kernel): 95.28%


In [40]:
model_accuracies = {name: accuracy for name, accuracy in results.items()}
cv_results = {}
for name, accuracy in sorted_results:
    model = classifiers[name]
    cv_mean, cv_std = cross_validate_model(name, model, x_tr_resample, y_tr_resample)
    cv_results[name] = (cv_mean, cv_std)


[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3434
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3538
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, t

[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000461 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3545
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3542
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3434
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000432 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3538
[LightGBM] [Info] Number of data points in the train set: 9784, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4892, number of negative: 4892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, t

All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ACER\anaconda3\envs\TA\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ACER\anaconda3\envs\TA\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\ACER\anaconda3\envs\TA\lib\site-packages\sklearn\naive_bayes.py", line 762, in fit
    self._count(X, Y)
  File "c:\Users\ACER\anaconda3\envs\TA\lib\site-packages\sklearn\naive_bayes.py", line 888, in _count
    check_non_negative(X, "MultinomialNB (input X)")
  File "c:\Users\ACER\anaconda3\envs\TA\lib\site-packages\s

: 

: 

In [41]:
print("\n--- Ranking Model Berdasarkan Cross-Validation ---")
sorted_cv_results = sorted(cv_results.items(), key=lambda x: x[1][0], reverse=True)

for rank, (name, (cv_mean, cv_std)) in enumerate(sorted_cv_results, 1):
    if cv_mean > 0:
        previous_accuracy = model_accuracies.get(name, 0)  # Akurasi awal
        improvement = (cv_mean - previous_accuracy) * 100  # Perubahan akurasi
        print(f"{rank}. {name}: {cv_mean * 100:.2f}% | (+{improvement:.2f}%)")


--- Ranking Model Berdasarkan Cross-Validation ---
1. CatBoost: 98.70% | (+0.88%)
2. Extra Trees: 98.67% | (+1.00%)
3. CatBoost L2: 98.66% | (+1.00%)
4. Hist Gradient Boosting: 98.52% | (+0.35%)
5. LightGBM: 98.50% | (+0.33%)
6. Random Forest: 98.48% | (+0.87%)
7. Label Spreading: 98.00% | (+1.60%)
8. Label Propagation: 97.98% | (+1.59%)
9. Multi-Layer Perceptron: 97.95% | (+1.40%)
10. LightGBM DART: 97.85% | (+0.34%)
11. Bagging Classifier: 97.82% | (+1.02%)
12. Gaussian Process: 97.24% | (+1.00%)
13. Radius Nearest Neighbors: 97.07% | (+1.19%)
14. K-Nearest Neighbors: 97.06% | (+1.63%)
15. Gradient Boosting: 96.72% | (+0.33%)
16. SVM (RBF Kernel): 96.69% | (+0.24%)
17. PyTorch Neural Network: 96.64% | (+0.80%)
18. Decision Tree: 96.52% | (+0.18%)
19. Extra Tree: 95.99% | (+1.02%)
20. SVM (Polynomial Kernel): 95.27% | (+0.61%)
21. AdaBoost: 93.71% | (+-0.09%)
22. Nu SVM: 89.93% | (+-3.42%)
23. SVM (Linear Kernel): 89.55% | (+0.57%)
24. Logistic Regression CV: 89.23% | (+0.66%)
25. Ca

: 

: 